<a href="https://www.kaggle.com/code/liaoguoying/unethology-predict-age-with-unet?scriptVersionId=172162636" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [11]:
!pip install pytorch-lightning qqq 
!pip install segmentation-models-pytorch qqq
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset, random_split
import torch
import torch.nn as nn
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from pytorch_lightning.loggers import WandbLogger
import matplotlib.pyplot as plt

In [ ]:
# Load the training dataset
train_data_path = "/kaggle/input/playground-series-s4e4/train.csv"
train_data = pd.read_csv(train_data_path)

# Display the first 5 rows of the training dataset
print(train_data.head())
print('#################################################################')

# Get basic information about the dataset, including data types and non-null values for each column
print(train_data.info())
print('#################################################################')

# Perform exploratory data analysis to understand the distribution of the data
print(train_data.describe())
print('#################################################################')


In [ ]:
# Load the data
data = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')

# Display basic information about the data
print("Initial Data Information:")
print(data.info())
print('#################################################################')

# 1. Check and handle records with height 0
zero_height = data[data['Height'] == 0]
print("Number of records with height 0:", zero_height.shape[0])
print('#################################################################')

# You can choose to delete these rows or replace them with mean/median
data = data[data['Height'] > 0]  # Delete these rows
# or
# median_height = data['Height'].median()
# data.loc[data['Height'] == 0, 'Height'] = median_height  # Replace with median

# 2. Encode categorical variables and ensure they are floating-point
data['Sex'] = data['Sex'].map({'M': 0.0, 'F': 1.0, 'I': 2.0})  # Label encoding to float
# Convert data type to float
data['Sex'] = data['Sex'].astype(float)

# 3. Data normalization
from sklearn.preprocessing import StandardScaler

# Define columns to be normalized
features_to_scale = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1', 'Whole weight.2', 'Shell weight']

# Initialize the scaler
scaler = StandardScaler()

# Normalize these columns
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

# Display a sample of the normalized data
print("Sample of normalized data:")
print(data[features_to_scale].head())

# Save the cleaned data to the specified folder
data.to_csv('/kaggle/working/cleaned_data.csv', index=False)
print("Data cleaning completed and saved to '/kaggle/working/cleaned_data.csv'.")

print("Data cleaning completed and saved to cleaned_data.csv")
print('#################################################################')

# Display basic information about the cleaned data
print("Cleaned Data Information:")
print(data.info())
print('#################################################################')

# Display the first 5 rows of the cleaned dataset
print("First 5 rows of cleaned dataset:")
print(data.head())
print('#################################################################')


In [7]:
# Display basic information about the data
print(data.info())  # Now we have reduced outliers and normalized the data
print('#################################################################')

# Display the first 5 rows of the cleaned dataset
print("First 5 rows of cleaned dataset:")
print(data.head())
print('#################################################################')


<class 'pandas.core.frame.DataFrame'>
Index: 90609 entries, 0 to 90614
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              90609 non-null  int64  
 1   Sex             90609 non-null  float64
 2   Length          90609 non-null  float64
 3   Diameter        90609 non-null  float64
 4   Height          90609 non-null  float64
 5   Whole weight    90609 non-null  float64
 6   Whole weight.1  90609 non-null  float64
 7   Whole weight.2  90609 non-null  float64
 8   Shell weight    90609 non-null  float64
 9   Rings           90609 non-null  int64  
dtypes: float64(8), int64(2)
memory usage: 7.6 MB
None
#################################################################
   id Sex  Length  Diameter  Height  Whole weight  Whole weight.1  \
0   0   F   0.550     0.430   0.150        0.7715          0.3285   
1   1   F   0.630     0.490   0.145        1.1300          0.4580   
2   2   I   0.160     0.110 

In [17]:
class AbaloneDataset(Dataset):
    """
    A custom dataset class for the Abalone dataset.
    """

    def __init__(self, dataframe):
        """
        Initialize the dataset with the provided DataFrame.

        Args:
            dataframe (pandas.DataFrame): The input DataFrame containing the Abalone data.
        """
        self.dataframe = dataframe

    def __len__(self):
        """
        Return the length of the dataset (number of samples).

        Returns:
            int: The length of the dataset.
        """
        return len(self.dataframe)

    def expand_and_fill(self, data):
        """
        Expand and fill the input data into a 32x32 single-channel matrix.

        Args:
            data (torch.Tensor): The input data with shape [feature_size], where feature_size=8.

        Returns:
            torch.Tensor: The expanded and filled data with shape [32, 32].
        """
        # Initialize a 32x32 output matrix
        output = torch.zeros((32, 32))

        # Fill each 4 rows with one feature
        for i in range(8):
            output[i * 4:(i + 1) * 4, :] = data[i]

        return output.unsqueeze(0)

    def __getitem__(self, idx):
        """
        Get a single data sample and its corresponding target value.

        Args:
            idx (int): The index of the sample in the dataset.

        Returns:
            tuple: A tuple containing the data (torch.Tensor) and target (torch.Tensor).
        """
        # Extract all features except 'id' and 'Rings'
        data = torch.tensor(
            self.dataframe.iloc[idx][1:-1].values.astype(np.float32)
        )  # Exclude id and Rings

        # Expand and fill the data into a 32x32 matrix
        data = self.expand_and_fill(data)

        # Get the target value (Rings)
        target = torch.tensor(self.dataframe.iloc[idx][-1], dtype=torch.float32)

        return data, target


# Load the data
dataframe = pd.read_csv('/kaggle/working/cleaned_data.csv')

# Create the dataset
dataset = AbaloneDataset(dataframe)

# Determine the size of the training and validation sets
val_size = int(0.2 * len(dataset))  # Validation set size is 20% of the dataset
train_size = len(dataset) - val_size  # Training set size is the remaining portion

# Randomly split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

# Check the output of the data loaders
for data, target in train_loader:
    print("Train batch - Data shape:", data.shape, "; Target shape:", target.shape)
    break

for data, target in val_loader:
    print("Validation batch - Data shape:", data.shape, "; Target shape:", target.shape)
    break


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = torch.tensor(self.dataframe.iloc[idx][-1], dtype=torch.float32)  # Rings作为目标
/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated

Train batch - Data shape: torch.Size([64, 8, 32, 32]) ; Target shape: torch.Size([64])


/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = torch.tensor(self.dataframe.iloc[idx][-1], dtype=torch.float32)  # Rings作为目标
/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = torch.tensor(self.dataframe.iloc[idx][-1], dtype=torch.float32)  # Rings作为目标
/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = torch.ten

Validation batch - Data shape: torch.Size([64, 8, 32, 32]) ; Target shape: torch.Size([64])


In [22]:
# Get the first batch of data from the train_loader
data_iter = iter(train_loader)
data_batch = next(data_iter)

# Unpack the batch data
inputs, targets = data_batch

# Print the shapes and some data points to confirm correct data loading
print("Input batch shape:", inputs.shape)
print("Target batch shape:", targets.shape)
print("First few inputs:", inputs[0])
print("First few targets:", targets[0])

# Select a sample input to visualize
data_to_show = inputs[0]

# Set up matplotlib plot
fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(20, 2.5))  # Create a 1x8 grid

# Plot the image
axes[0].imshow(data_to_show[0], cmap='gray')
axes[0].set_title('Image')

plt.tight_layout()
plt.show()


/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = torch.tensor(self.dataframe.iloc[idx][-1], dtype=torch.float32)  # Rings作为目标
/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = torch.tensor(self.dataframe.iloc[idx][-1], dtype=torch.float32)  # Rings作为目标
/tmp/ipykernel_33/2664565292.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = torch.ten

tensor([[1.1621, 1.1621, 1.1621,  ..., 1.1621, 1.1621, 1.1621],
        [1.1621, 1.1621, 1.1621,  ..., 1.1621, 1.1621, 1.1621],
        [1.1621, 1.1621, 1.1621,  ..., 1.1621, 1.1621, 1.1621],
        ...,
        [1.1621, 1.1621, 1.1621,  ..., 1.1621, 1.1621, 1.1621],
        [1.1621, 1.1621, 1.1621,  ..., 1.1621, 1.1621, 1.1621],
        [1.1621, 1.1621, 1.1621,  ..., 1.1621, 1.1621, 1.1621]]) tensor([[-2.0480, -2.0480, -2.0480,  ..., -2.0480, -2.0480, -2.0480],
        [-2.0480, -2.0480, -2.0480,  ..., -2.0480, -2.0480, -2.0480],
        [-2.0480, -2.0480, -2.0480,  ..., -2.0480, -2.0480, -2.0480],
        ...,
        [-2.0480, -2.0480, -2.0480,  ..., -2.0480, -2.0480, -2.0480],
        [-2.0480, -2.0480, -2.0480,  ..., -2.0480, -2.0480, -2.0480],
        [-2.0480, -2.0480, -2.0480,  ..., -2.0480, -2.0480, -2.0480]]) tensor([[-2.0065, -2.0065, -2.0065,  ..., -2.0065, -2.0065, -2.0065],
        [-2.0065, -2.0065, -2.0065,  ..., -2.0065, -2.0065, -2.0065],
        [-2.0065, -2.0065, -

In [23]:
def rmsle(y_pred, y_true):
    """
    Calculate the Root Mean Squared Logarithmic Error (RMSLE).

    Args:
        y_pred (torch.Tensor): The predicted outputs from the model, should be a tensor.
        y_true (torch.Tensor): The true target values, should be a tensor of the same shape as y_pred.

    Returns:
        torch.Tensor: A scalar tensor representing the RMSLE for the current batch.
    """
    # Ensure predictions and targets are greater than zero after adding one to avoid log(0)
    log_pred = torch.log1p(y_pred)
    log_true = torch.log1p(y_true)

    # Calculate the squared differences between the two
    squared_log_error = (log_pred - log_true) ** 2

    # Mean the squared differences then take the square root
    mean_squared_log_error = torch.mean(squared_log_error)
    rmsle = torch.sqrt(mean_squared_log_error)

    return rmsle

# Test it with a simple tensor 😝
y_pred = torch.tensor([3.0, 5.0, 2.5], dtype=torch.float32)
y_true = torch.tensor([2.0, 4.0, 3.0], dtype=torch.float32)

# Calculate the RMSLE
loss = rmsle(y_pred, y_true)
print("RMSLE Loss:", loss.item())


In [24]:
class AbaloneModel(pl.LightningModule):
    """
    U-Net based regression model for Abalone age prediction.
    """

    def __init__(self):
        super(AbaloneModel, self).__init__()

        # Encoder-Decoder architecture using U-Net
        self.model = smp.Unet(
            encoder_name="mit_b1",  # Select encoder (e.g., ResNet34)
            # encoder_weights="imagenet",  # Use pretrained weights
            in_channels=3,
            classes=1,  # Output a single age prediction
            decoder_attention_type='scse',  # No activation function or whatever you like
        )

        # Regressor head to output age prediction
        self.regressor = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),  # Pool to 1x1
            nn.Flatten(),  # Flatten
            nn.Linear(1, 1),  # Fully connected layer
        )

    def forward(self, x):
        # Repeat input channel to match U-Net input (3 channels)
        x = x.repeat(1, 3, 1, 1)

        # Pass through U-Net encoder-decoder
        x = self.model(x)

        # Apply regressor head for age prediction
        x = self.regressor(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)  # Model output should be [batch_size, 1]
        y_hat = y_hat.squeeze(-1)  # Remove last dim, shape becomes [batch_size]

        loss = rmsle(y_hat, y)

        # Log training loss
        self.log("train_loss", loss, on_step=True, on_epoch=False, logger=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)  # Model predictions on validation set
        y_hat = y_hat.squeeze(-1)  # Ensure predictions are [batch_size]

        # Calculate loss
        loss = rmsle(y_hat, y)

        # Optionally: Log validation loss
        self.log("val_loss", loss, on_step=True, on_epoch=True, logger=True, prog_bar=True)

        # return loss

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=1e-3)


In [ ]:
# wandb_logger = WandbLogger(project="Abalone", name="Round4")

model = AbaloneModel()
trainer = pl.Trainer(max_epochs=40, logger=None)

# Go train that shxt
trainer.fit(model, train_loader, val_loader)

# **WTF IT WORKS?**😝😝😝😝😝😝😝😝😝

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Freeze the model to ensure parameters don't change (in PyTorch Lightning)
model.freeze()

# Create a DataFrame to store results
results = pd.DataFrame()

# Initialize counters for overall accuracy
accurate_predictions_count = 0
total_predictions_count = 0

# Disable gradient computation for inference
with torch.no_grad():
    for batch in val_loader:  # Assuming val_loader is your validation set loader
        inputs, targets = batch
        # inputs = inputs.to('cuda')  # Move input data to GPU (if using GPU)

        predictions = model(inputs)  # Get model predictions
        predictions = predictions.squeeze(-1)  # Adjust shape of predictions if needed

        # Convert data to CPU and NumPy
        predictions = predictions.cpu().numpy()
        targets = targets.cpu().numpy()

        # Calculate accuracy: predictions within 1 year of actual age
        accurate_predictions = np.abs(predictions - targets) <= 1
        accurate_predictions_count += np.sum(accurate_predictions)
        total_predictions_count += len(predictions)

        # Add results to DataFrame
        batch_results = pd.DataFrame({
            "Predicted Age": predictions,
            "Actual Age": targets,
            "Accurate": accurate_predictions
        })
        results = pd.concat([results, batch_results], ignore_index=True)

# Calculate overall accuracy
overall_accuracy = accurate_predictions_count / total_predictions_count * 100
print(f"Overall Accuracy: {overall_accuracy:.2f}%")

# Display or analyze results
print(results.head())

# Optionally save results to CSV
results.to_csv("prediction_results.csv", index=False)
